In [3]:
pip install transformers sentence-transformers numpy

In [4]:
import numpy as np
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

# -------------------------------
# 1. Carregar documentos (exemplo)
# -------------------------------
documents = {
    "doc1.txt": "O Python é uma linguagem de programação popular para ciência de dados.",
    "doc2.txt": "Modelos de linguagem como BERT e T5 são amplamente usados em NLP.",
    "doc3.txt": "RAG significa Retrieval Augmented Generation, uma técnica que combina recuperação com geração.",
}

doc_texts = list(documents.values())

# -------------------------------
# 2. Gerar embeddings
# -------------------------------
embedder = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = embedder.encode(doc_texts)

# -------------------------------
# 3. Recuperação via similaridade de cosseno
# -------------------------------
def retrieve(query, top_k=1):
    query_emb = embedder.encode(query)
    scores = util.cos_sim(query_emb, doc_embeddings)[0]
    top_results = scores.topk(k=top_k)
    best_docs = [doc_texts[idx] for idx in top_results.indices]
    return best_docs

# -------------------------------
# 4. LLM (QA Pipeline)
# -------------------------------
qa = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

# -------------------------------
# 5. Função RAG
# -------------------------------
def rag(query):
    context = retrieve(query, top_k=1)[0]

    answer = qa({
        "question": query,
        "context": context
    })

    return {
        "query": query,
        "contexto_usado": context,
        "resposta": answer["answer"]
    }

# -------------------------------
# 6. Teste
# -------------------------------
result = rag("O que é RAG?")
print(result)

config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/question_answering.py:395: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


{'query': 'O que é RAG?', 'contexto_usado': 'RAG significa Retrieval Augmented Generation, uma técnica que combina recuperação com geração.', 'resposta': 'significa Retrieval Augmented Generation'}
